## Experiment 1. Predicting diagnosis from sMRI features

In this experiment, we try to predict diagnosis group based on features extracted from anatomical MR images using FreeSurfer.

Loads the Destrieux atlas FreeSurfer statistic outputs and then restricts it to cortical thickness or volume measures. Subcortical volume measures are always included. We include the covariates listed in Table X. For each categortical covariate under the One-hot encoding column, we create a separate feature column for each value of the feature. Each column contains a 1 wherever the feature is present and 0 otherwise. Other covariates are included as is. Redudant columns in the one hot encoded matrix are removed. The effect of the covariates on the sMRI data is removed via a projection matrix. We create the projection matrix $P$ by applying the following transformations:

$$P = I - C(C^{T}C)^{-1}C^{T}$$

$$X^{*} = PX$$

Where $I$ is the identity matrix of the $N x P$ covariate matrix, $C$ is the covariate matrix and $X^{*}$ is the new data matrix with the effect of covariates removed.

In [1]:
import os
import numpy as np
import pandas as pd
from custom import utils


# VALIDATION DATA
valbase = '/storage/gablab001/data/genus/current/structured/validate'

# 183_cor_subcor_D_thickness.csv contains the 170 features for the validation set
valdatacob = '183_cor_subcor_D_thickness.csv'
valdatacob = pd.read_csv(os.path.join(valbase, valdatacob))

# the file containing the covariates
valcov = pd.read_csv(os.path.join(valbase, '183_covariates_to_use.csv'))

# this is y in the classification analysis
valres = [1 if i == 0 else 0 for i in valcov['diag'].values]

valcov = valcov[['ICV','age','gender']]

# base directory where some text files live, they incude header names
# example: example_header.txt will have:
# pallidum
# cerebellum
txtb = '/storage/gablab001/data/genus/current/structured/genus/text_files_for_indexing'

# directory to where the genus brain that is
bd = '/storage/gablab001/data/genus/current/structured/brain/'

# the header text file for the 170 columns, this will be used to subset the 
# entire genus data
thickness = np.genfromtxt(os.path.join(txtb, '170_columns.txt'), dtype=str)

# making equivalent volume headers from the thickness headers
volume = ' '.join(thickness).replace('thickness_D','volume_D').split(' ')

# this variable is for convenience so that i dont hav to change
# where the headers are in multiple places, just here
colheads = thickness

# loading the covariate headers that will be one hot encoded
cvar_encode = np.genfromtxt(os.path.join(txtb, 'covars_ecn.txt'), dtype=str)

# the covariate headers that wont be one hot encoded
cvar = np.genfromtxt(os.path.join(txtb, 'covars_no_ecn.txt'), dtype=str)

# GENUS brain data
brain = pd.read_csv(os.path.join(bd, 'GENUS_FS_ATLAS_D.csv'), low_memory=False)

# GENUS response variable
response = brain[['IID','GROUP']]

# here i combined all the needed data so that I can drop rows all together and 
# make sure all parts of the data, brain regions, covariates, ID, response are
# sorted by the same rows
combined = pd.concat([
    brain[colheads],
    brain[cvar],
    brain[cvar_encode],
    brain[['IID','GROUP']]
], axis=1).dropna().drop_duplicates('IID')

# the genus y in the classification analysis
response = combined['GROUP'].values

# subsetting genus data to only include the 170 features
X_data = combined[colheads].reset_index(drop=True)

# getting the covariates that wont be one hot encoded
cvar_ne = combined[cvar].reset_index(drop=True)

# and the covariates that will be one hot encoded
cvar_e = combined[cvar_encode].reset_index(drop=True)

# performing one hot encoding
cvar_e = pd.concat([
    pd.DataFrame(utils.encoder(cvar_e[col])) for col in cvar_e.columns
], axis=1, ignore_index=True)

# recombining covariates
cvars = pd.concat([cvar_ne, cvar_e], axis=1)

# response variable to be fed into classifier
y = np.array([1 if i == 'Schizophrenia' else 0 for i in response])

# remove effects of covariates
# NOTE: this will result in extremely poor results for the validation set
#       because the same covariates cannot be applied to the validation set
#       it's only when the same covariates are removed from each set that we see
#       results similar to what I show
non_sing_cvars = utils.make_non_singular(cvars.values)
XG = utils.proj(X_data.values, non_sing_cvars)
XCO = utils.proj(valdatacob.values, valcov.values)

# dictionary that will be passed to the classifier node
data_dict = {'GENUS_X':XG, 'GENUS_y':y,
             'GENUS_XCOLS':X_data.columns.values,
             'GENUS_COVCOLS':cvars.columns.values,
             'COBREFMRI_X': XCO, 'COBREFMRI_y': valres,
             'COBREFMRI_XCOLS': valdatacob.columns.values,
             'COBREFMRI_COVCOLS': valcov.columns.values}

To test how well the brain features predict the diagnosis we employ a nested cross validation scheme. The model of choice is Logistic Regression with and L1 penalty to impose sparcity on the feature space. We picked an L1 penalty because we want to find a subset of the feature space that best predicts the outcome. The best regularization parameter was chosen in the inner loop of the nested cross validation scheme and then it was applied to the other loop. This paramter would determind the features which would be considered in the model at the time of classification. Two cross validators(CV) were chosen, for the outer loop a stratisfied shuffle split was selection, this CV was run over 400 iterations. It randomly sampled from the data in a stratifed manner which preserves the relative perctanges of patients and controls in the entire sample. The inner CV was stratified k-fold, where k = 15. 

The preprocessing steps before the logistic regression model was applied to the data were demeaning by subtracting the feature mean from each observation in that feature, and scaling by dividing each observation within the feature by the feature standard deviation (after demeaning). The accuracy measured we used was Receiver Operating Characteristic Area Under the Curve. 

In [76]:
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier

In [57]:
cv_in = StratifiedKFold(n_splits=15, shuffle=True)
cv_out = StratifiedShuffleSplit(n_splits=400)

clf = Pipeline([
    ('scale', StandardScaler()),
    ('lg', linear_model.LogisticRegressionCV(
        penalty='l1',
        solver='liblinear',
        cv=cv_in,
        Cs=200,
        n_jobs=-1
    ))
])


clf_rf = Pipeline([
    ('scale', StandardScaler()),
    ('lg', RandomForestClassifier(n_estimators=200))
])

In [6]:
results = {}
for idx, (train, test) in enumerate(cv_out.split(data_dict['GENUS_X'], data_dict['GENUS_y'])):
    
    clf.fit(data_dict['GENUS_X'][train], data_dict['GENUS_y'][train])
    
    results['coef_{}'.format(idx)] = clf.named_steps['lg'].coef_
    
    results['test_auc_split_{}'.format(idx)] = roc_auc_score(
                                             data_dict['GENUS_y'][test], 
                                             clf.predict(data_dict['GENUS_X'][test])
                                         )
    results['train_auc_split_{}'.format(idx)] = roc_auc_score(
                                              data_dict['GENUS_y'][train],
                                              clf.predict(data_dict['GENUS_X'][train])
                                         )
    
    

Given that the model used applied an L1 penalty this allowed us to get a view at each of the 400 outer cross validation iterations which features from the input sMRI matrix were included in the model. We aggregated these results and created a stability heat map over the brain. The darker the color of a region, the more times a feature was included in the logistic regression model across the CV 400 iterations. 

In [8]:
% matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from collections import Counter
sns.set_style('whitegrid')

def nonzero(x):
    return np.nonzero(x[0])[0]

def save_pickle(x, name):
    with open(name, "wb") as save:
        pickle.dump(x, save, protocol=pickle.HIGHEST_PROTOCOL)
    return None

test_auc = [val for key, val in results.items() if 'test_auc_split' in key]
train_auc = [val for key, val in results.items() if 'train_auc_split' in key]

aucdf = pd.DataFrame({'Test_AUC': [val for key, val in results.items() if 'test_auc_split' in key],
                      'Train_AUC': [val for key, val in results.items() if 'train_auc_split' in key]})

melted = pd.melt(aucdf, var_name='Partition', value_name='ROC AUC SCORE')

ancc = [inner for outer in [data_dict['GENUS_XCOLS'][nonzero(results['coef_{}'.format(idx)])]
        for idx in range(400)] for inner in outer]

region_count = dict(Counter(ancc))

lh = {key.replace('lh_', '').replace('.', '-').replace('_thickness_D', ''):val for key, 
      val in region_count.items() if 'lh' in key}
rh = {key.replace('rh_', '').replace('.', '-').replace('_thickness_D', ''):val for key, 
      val in region_count.items() if 'rh' in key}